In [1]:
from random import choices, randint, randrange, random
from typing import List, Optional, Callable, Tuple
import numpy as np
import pandas as pd
import sobol_seq
import matplotlib.pyplot as plt

In [2]:
Genome = List[int]
Population = List[Genome]
FitnessFunc = Callable[[Genome], int]

Problem 1

In [3]:
n_point = 20
k_max = 250
dim = 2

def objective_function(X):
    f1 = X[0]+X[1]**2 #-2*x
    f_list = [f1]
    denom = 1
    for f in f_list:
        denom +=np.abs(f)
    F = 1/denom
    return F

boundaries = np.array([(-10,10) for _ in range (dim)])

In [4]:
min_value = boundaries.min()
max_value = boundaries.max()
num_bits = 64  # Number of bits for each number

In [5]:
"""GENERATE POINTS USING SOBOL SEQUENCE"""
def generate_points(dim,npoint,low=-10,high=10):
    if type(low) != type(high):
        raise TypeError('The type of "low" and "high" should be the same.')
    if type(low) == int:
        boundaries = [(low,high) for _ in range (dim)]
    elif type(low) == list or type(low) == np.ndarray:
        if len(low) != len(high):
            raise TypeError('The length of "low" and "high" should be the same.')
        else:
            boundaries = [(low[i],high[i]) for i in range (len(low))]

    # Generate Sobol sequence points
    sobol_points = sobol_seq.i4_sobol_generate(dim, npoint)

    # Scale the Sobol points to fit within the specified boundaries
    scaled_points = []
    for i in range(dim):
        a, b = boundaries[i]
        scaled_dim = a + sobol_points[:, i] * (b - a)
        scaled_points.append(scaled_dim)

    # Transpose the scaled points to get points per dimension
    scaled_points = np.array(list(map(list, zip(*scaled_points))))
    return scaled_points

In [6]:
iter_points = generate_points(dim,n_point,boundaries[:,0],boundaries[:,1])
iter_points

array([[ 0.   ,  0.   ],
       [ 5.   , -5.   ],
       [-5.   ,  5.   ],
       [-2.5  , -2.5  ],
       [ 7.5  ,  7.5  ],
       [ 2.5  , -7.5  ],
       [-7.5  ,  2.5  ],
       [-6.25 , -3.75 ],
       [ 3.75 ,  6.25 ],
       [ 8.75 , -8.75 ],
       [-1.25 ,  1.25 ],
       [-3.75 , -6.25 ],
       [ 6.25 ,  3.75 ],
       [ 1.25 , -1.25 ],
       [-8.75 ,  8.75 ],
       [-8.125, -0.625],
       [ 1.875,  9.375],
       [ 6.875, -5.625],
       [-3.125,  4.375],
       [-0.625, -8.125]])

In [7]:
def encode_number(number, min_value=-10, max_value=10, num_bits=32):
    # Normalize the number to a value between 0 and 1
    normalized = (number - min_value) / (max_value - min_value)
    # Convert it to an integer representation
    int_representation = int(normalized * (2**num_bits - 1))
    # Convert the integer to binary and pad with zeros
    return [int(x) for x in format(int_representation, '0{}b'.format(num_bits))]

def decode_number(genome:Genome, min_value=-10, max_value=10, num_bits=32):
    # Convert the binary string to an integer
    int_representation = int(''.join(map(str, genome)), 2)
    # Scale down to the normalized value
    normalized = int_representation / (2**num_bits - 1)
    # Denormalize to get the real number
    return min_value + normalized * (max_value - min_value)

def encode_list(number_list,min_value=-10, max_value=10, num_bits=32):
    encoded = []
    for number in number_list:
        encoded += encode_number(number,min_value, max_value, num_bits)
    return encoded

def decode_list(encoded_list,min_value=-10, max_value=10, num_bits=32):
    numbers = []
    for i in range(0, len(encoded_list), num_bits):
        binary_list = encoded_list[i:i + num_bits]
        number = decode_number(binary_list, min_value, max_value, num_bits)
        numbers.append(number)
    return numbers


In [28]:
def generate_population(set_of_points:np.ndarray,min_value=-10, max_value=10, num_bits=32) -> Population:
    return [encode_list(set_of_points[point],min_value, max_value, num_bits) for point in range(len(set_of_points))]

def single_point_crossover(a: Genome, b: Genome, print_cutoff=False) -> Tuple[Genome, Genome]:
    if len(a) != len(b):
        raise ValueError("Genomes a and b must be of same length")
    length = len(a)
    # if the length less than 2, then there is no point to do the function
    if length < 2:  
        return a, b
    # generate random number as the cutoff of the crossover
    p = randint(1, length - 1)
    if print_cutoff == True:
        print(p)
    
    return a[0:p] + b[p:], b[0:p] + a[p:]

def mutation(genome: Genome, num: int = 1, probability: float = 0.5) -> Genome:
    # num: generate how many chromosome(s) that we want to mutate
    for _ in range(num):
        # index sets which chromosome we want to change
        index = randrange(len(genome))
        # the change algorithm
        genome[index] = genome[index] if random() > probability else abs(genome[index] - 1)
    return genome

def fitness_function(genome: Genome, min_value=-10, max_value=10, num_bits=32) -> float:
    X = decode_list(genome, min_value, max_value, num_bits)
    return objective_function(X)

def population_fitness(population: Population, fitness_func: FitnessFunc, min_value=-10, max_value=10, num_bits=32) -> float:
    return sum([fitness_func(genome,min_value, max_value, num_bits) for genome in population])

def selection_pair(population: Population, fitness_func: FitnessFunc,min_value=-10, max_value=10, num_bits=32) -> Population:
    return choices(
        population=population,
        weights=[fitness_func(gene) for gene in population],
        k=2
    )
def sort_population(population: Population, fitness_func: FitnessFunc,min_value=-10, max_value=10, num_bits=32) -> Population:
    return sorted(population, key=lambda x: fitness_func(x, min_value, max_value, num_bits), reverse=True)

def genome_to_string(genome: Genome) -> str:
    return "".join(map(str, genome))

def print_stats(population: Population, generation_id: int, fitness_func: FitnessFunc,min_value=-10, max_value=10, num_bits=32,binary_mode=False):
    print("GENERATION %02d" % generation_id)
    print("=============")
    if binary_mode == True:
        print("Population: [%s]" % ", ".join([genome_to_string(gene) for gene in population]))
        print("Avg. Fitness: %f" % (population_fitness(population, fitness_func,min_value, max_value, num_bits) / len(population)))
        sorted_population = sort_population(population, fitness_func,min_value, max_value, num_bits)
        print(
            "Best: %s (%f)" % (genome_to_string(sorted_population[0]), fitness_func(sorted_population[0],min_value, max_value, num_bits)))
        print("Worst: %s (%f)" % (genome_to_string(sorted_population[-1]),
                                fitness_func(sorted_population[-1],min_value, max_value, num_bits)))
        print("")
    else:
        print(f"Population: {[decode_list(population[i],num_bits=num_bits) for i in range (len(population))]}")
        print("Avg. Fitness: %f" % (population_fitness(population, fitness_func,min_value, max_value, num_bits) / len(population)))
        sorted_population = sort_population(population, fitness_func,min_value, max_value, num_bits)
        print(f"Best: {(decode_list(population[0],num_bits=num_bits))}")
        print(f"Worst: {(decode_list(population[1],num_bits=num_bits))}")
        print("")
    # return sorted_population[0]

In [9]:
popA = generate_population(iter_points,num_bits=num_bits)
print(popA[0])
print(popA[1])
decoded_genome = [decode_list(popA[i],num_bits=num_bits) for i in range (len(popA))]
print(decoded_genome)

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[0.0, 0.0], [5.0, -5.0], [-5.0, 5.0], [-2.5, -2.5], [7.5, 7.5], [2.5, -7.5], [-7.5, 2.5], [-6.25, -3.75], [3.75, 6.25], [8.75, -8.75], [-1.25, 1.25], [-3.75, -6.25], [6.25, 3.75], [1.25, -1.25], [-8.75, 8.75], [-8.125, -0.625], [

In [10]:
population_fitness(population=popA,fitness_func=fitness_function,num_bits=num_bits)

0.011017386813565157

In [11]:
print(selection_pair(popA,fitness_func=fitness_function,num_bits=num_bits))

[[0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]


In [15]:
# Instead of 'sorted', use a different name for the variable, like 'sorted_population'
sorted_population = sort_population(popA, fitness_func=fitness_function, num_bits=64)

for individual in sorted_population:
    print(fitness_function(individual, num_bits=64))

print(sorted_population)


1.0
0.7619047619047619
0.4444444444444444
0.26229508196721313
0.21052631578947367
0.11449016100178891
0.11347517730496454
0.05876951331496786
0.047619047619047616
0.0469208211143695
0.03225806451612903
0.027538726333907058
0.02530644523527086
0.02282453637660485
0.016736401673640166
0.015444015444015444
0.015062367615909626
0.014532243415077202
0.011585807385952208
0.011017386813565157
[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0

In [29]:
print_stats(popA,1,fitness_func=fitness_function,num_bits=64)

GENERATION 01
Population: [[0.0, 0.0], [5.0, -5.0], [-5.0, 5.0], [-2.5, -2.5], [7.5, 7.5], [2.5, -7.5], [-7.5, 2.5], [-6.25, -3.75], [3.75, 6.25], [8.75, -8.75], [-1.25, 1.25], [-3.75, -6.25], [6.25, 3.75], [1.25, -1.25], [-8.75, 8.75], [-8.125, -0.625], [1.875, 9.375], [6.875, -5.625], [-3.125, 4.375], [-0.625, -8.125]]
Avg. Fitness: 0.162638
Best: [0.0, 0.0]
Worst: [5.0, -5.0]

